# Boosting

In general boosting is an ensemble method which aims to combine
the output of many *weak* learners in order to produce a single 
*strong* learner in relation to some objective function. In this
context a *weak* learner will be only slightly correlated with a
true classification i.e marginally better than random guessing.
(NB boosting can be extended to regression). 

# AdaBoost

The AdaBoost algorithm solves a two class problem with 
features $\boldsymbol{x}_{i}\in\mathbb{R}^{d}$ and outputs $y_{i}\in\{-1,1\}$, for $i=1,2,\ldots,n$. We consider a 
weak classifier $G(x)\}$ and an error rate
$$
E(G,\boldsymbol{x},y) = \frac{\sum_{i=1}^{N}\bigl(1-\delta_{y_{i},G(\boldsymbol{x}_{i})}\bigr)}{N}
$$

The algorithm sequentially applies this classifier to a modified
version of the data $\boldsymbol{x}$. This produces the sequence 
$\{G_{m}(\boldsymbol{x})\}_{m=1}^{M}$ of classifiers. The data 
modification, at step $k$, is a weighting applied to each instance 
$(\boldsymbol{x}_{i},y_{i})$ depending on wether the previous 
classifier, $G_{k-1}(x)$, correctly or incorrectly classified the 
instance. If classification was correct the weight decreases, and if 
incorrect the weight increases, with the initial weighting $\frac{1}{N}$ for all the instances. Additionally a weight of
$\alpha_{k}$ is computed bases on the error 
$E(G_{k},\boldsymbol{x},y)$ which determines the contribution of 
$G_{k}$ to the final classifier.

In detail the *discrete* algorithm is as follows

1. Set weights to $w_{i}=\frac{1}{N}$ $\forall$ $i$. 
2. For $m=1$ to $M$.
    2. Fit $G_{m}$ to training data with weightings $w_{i}$.
    2. Compute the weighted error 
    $$
    E_{m} = \frac{\sum_{i=1}^{N}w_{i}(1-\delta_{y_{i},G_{m}(\boldsymbol{x}_{i})})}{\sum_{i=1}^{N}w_{i}}
    $$
    2. Compute the final weight 
    $$
        \alpha_{i} = \log\frac{1-E_{m}}{E_{m}}
    $$
    2. Update the weight for the instances
    $$
        w_{i} \leftarrow w_{i}e^{\alpha_{m}(1-\delta_{y_{i},G_{m}(\boldsymbol{x}_{i})})}, i=1,2,\ldots,N
    $$
3. Output the final classifier
    $$
        G(x) = \text{sgn}\biggl(\sum_{m=1}^{M}\alpha_{m}G_{m}(x)\biggr)
    $$

## AdaBoost for decision trees

In the decision tree context an example of a weak classifier is
the decision stump, which includes one rule at the root node
and two resultant leaf nodes. Using a decision stump may 
yield error rates of $\sim 45\%$ but applying AdaBoost (with 
$G$ the decision stump) can improve this to $\sim 6\%$ after 
400 iterations.

## Julia Implementations

### Libraries
    
- DecisionTree.jl https://github.com/bensadeghi/DecisionTree.jl (Decision Stumps)

## References

[1] The Elements of Statistical Learning (Ch 10.1)

# Boosting as an additive expansion

Boosting can be thought of as fitting an additive expansion 
$$
    f(x) = \sum_{m=1}^{M}\beta_{m}b(x:\gamma_{m})
$$

Where the $\beta_{m}, m=1,2,\ldots,M$ are expansion coefficients 
and $b(x:\gamma)\in\mathbb{R}$ are *basis* functions. For AdaBoost
the basis functions are the classifiers $G_{m}$. In this context a 
forward stagewise additive modeling algorithm may be constructed 
to produce more accurate approximations of $f$, this involves
an iterative optimisation
$$
    (\beta_{m},\gamma_{m}) = \text{argmin}_{\beta,\gamma}\sum_{i=1}^{N}L((y_{i},f_{m-1}(x_{i})+\beta b(x_{i}:\gamma))
$$

such that

$$
\begin{align*}
    f_{m}(x) &= f_{m-1}(x)+\beta_{m}b(x:\gamma_{m})\\
    f_{0}(x) &= 0
\end{align*}
$$

where $L$ is some loss function. In this context AdaBoost is defined
by using the loss
$$
    L(y,f(x)) = e^{-yf(x)}
$$

# Gradient Boosting

Gradient boosting is a boosting algorithm which allows for the use 
of an arbirary (but differentiable) loss function. The intuition
is to think of a boosting algorithm as an optimisation procedure in
the form of stagewise additive modelling, but unlike the AdaBoost
example above *weaknesses* are identified by negative gradients not by 
high-weight data points.


The genral algorithm for an arbitrary differentiable loss,
$L$, with parameters $\gamma$ is as follows

1. Set 
$$
    f_{0}(x)=\text{argmin}_{\gamma}\sum_{i=1}^{N}L(y_{i},\gamma)
$$
2. For $m=1$ to $M$
    2. For $i=1,2,\ldots,N$ compute
    $$
        r_{im} = -\biggl[\frac{\partial L(y_{i},f(x_{i}))}{\partial f(x_{i})}\biggr]_{f=f_{m-1}}
    $$
    2. Fit learner, $h_{m}(x)$ to the training set $\{(x_{i},r_{im})\}_{i=1}^{N}$
    2. Compute
    $$
        \gamma_{m} = \text{argmin}_{\gamma}\sum_{i=1}^{N}L(y_{i},f_{m-1}(x_{i})+\gamma h_{m}(x))
    $$
    2. Update the model
    $$
        f_{m}(x) = f_{m-1}(x) +\gamma_{m}h_{m}(x_{i})
    $$
3. Output the final model
$$
    \hat{f}(x) = f_{M}(x)
$$

## Gradient Boosted CART

If a tree model is used (CART is often gradient boosted), in step
B tree models are fitted to the targets producing terminal regions
$R_{jm}, j=1,2,\ldots,J_{m}$, then C becomes
$$
    \gamma_{jm} = \text{argmin}_{\gamma}\sum_{x_{i}\in R_{jm}}L(y_{i},f_{m-1}(x_{i})+\gamma)
$$

and finally D becomes 
$$
        f_{m}(x) = f_{m-1}(x) +\sum_{j=1}^{J_{m}}\gamma_{jm}\boldsymbol{1}_{R_{jm}}(x)
$$

## Julia Implementations

### Libraries
    
- https://github.com/dmlc/XGBoost.jl
- https://github.com/svs14/GradientBoost.jl

## References

[1] The Elements of Statistical Learning (Ch 10.2, 10.4, 10.9, and 10.10)

[2] Friedman, Jerome H, Greedy function approximation: a gradient boosting machine, Annals of statistics 2001